<a href="https://colab.research.google.com/github/MarioPasc/GNN/blob/main/GNN_tnbc_genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List, Dict, Tuple
import requests
import pandas as pd
import networkx as nx
import torch
from torch_geometric.data import Data
from bravado.client import SwaggerClient

# -------------------------------
# 1. Obtener Red PPI con STRINGdb
# -------------------------------

def get_stringdb_interactions(genes: List[str], species: int = 9606, score_threshold: int = 700) -> pd.DataFrame:
    """
    Obtiene las interacciones proteína-proteína (PPI) para una lista de genes desde STRINGdb.

    Args:
        genes (List[str]): Lista de genes/proteínas.
        species (int): ID de especie (9606 para humanos).
        score_threshold (int): Umbral de score de confianza (0-1000).

    Returns:
        pd.DataFrame: DataFrame con pares de proteínas y su score de interacción.
    """
    url = "https://string-db.org/api/json/network"
    params = {
        "identifiers": "%0d".join(genes),
        "species": species,
        "required_score": score_threshold,
        "limit": 1000
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    interactions = response.json()

    # Procesar resultados
    data = [{
        "protein1": entry["preferredName_A"],
        "protein2": entry["preferredName_B"],
        "score": entry["score"]
    } for entry in interactions]
    return pd.DataFrame(data)

# -------------------------------
# 2. Obtener Vecinos de STRINGdb
# -------------------------------

def get_stringdb_neighbors(genes: List[str], limit: int = 30, species: int = 9606) -> List[str]:
    """
    Expande la red obteniendo vecinos adicionales de STRINGdb.

    Args:
        genes (List[str]): Lista de genes/proteínas iniciales.
        limit (int): Máximo número de vecinos por gen.
        species (int): ID de especie (9606 para humanos).

    Returns:
        List[str]: Lista expandida de genes incluyendo vecinos.
    """
    url = "https://string-db.org/api/json/network/expand"
    expanded_genes = set(genes)
    for gene in genes:
        params = {
            "identifiers": gene,
            "species": species,
            "limit": limit
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        neighbors = response.json()
        for entry in neighbors:
            expanded_genes.add(entry["preferredName_B"])
    return list(expanded_genes)

# -------------------------------
# 3. Extraer Datos de Expresión de TCGA
# -------------------------------

def get_tcga_expression_data(genes: List[str], study_id: str = "brca_tcga") -> pd.DataFrame:
    """
    Obtiene datos de expresión génica desde cBioPortal para los genes especificados.

    Args:
        genes (List[str]): Lista de genes de interés.
        study_id (str): ID del estudio de TCGA.

    Returns:
        pd.DataFrame: DataFrame con la expresión génica de cada gen por muestra.
    """
    # Inicializar cliente cBioPortal
    cbioportal = SwaggerClient.from_url(
        'https://www.cbioportal.org/api/v2/api-docs',
        config={"validate_requests": False, "validate_responses": False, "validate_swagger_spec": False}
    )

    # Obtener IDs de los genes
    def get_gene_ids(gene_symbols: List[str]) -> Dict[str, int]:
        response = cbioportal.Genes.fetchGenesUsingPOST(geneIdType="HUGO_GENE_SYMBOL", geneIds=gene_symbols).result()
        return {gene.hugoGeneSymbol: gene.entrezGeneId for gene in response}

    gene_ids = get_gene_ids(genes)
    profile_id = "brca_tcga_rna_seq_v2_mrna"

    # Extraer datos
    filter_data = {
        'molecularProfileIds': [profile_id],
        'entrezGeneIds': list(gene_ids.values())
    }
    expression_data = cbioportal.Molecular_Data.fetchMolecularDataInMultipleMolecularProfilesUsingPOST(
        molecularDataMultipleStudyFilter=filter_data
    ).result()

    # Formatear resultados
    data = [{
        "sample": d.sampleId,
        "gene": str(d.entrezGeneId),
        "value": float(d.value)
    } for d in expression_data if d.value not in [None, "NA"]]
    df = pd.DataFrame(data)
    df["gene"] = df["gene"].map({str(v): k for k, v in gene_ids.items()})
    return df.pivot(index="sample", columns="gene", values="value")

# -------------------------------
# 4. Formato para PyTorch Geometric
# -------------------------------

def create_pyg_data(ppi_df: pd.DataFrame, expression_df: pd.DataFrame) -> Data:
    """
    Crea un objeto Data de PyTorch Geometric a partir de la red PPI y datos de expresión.

    Args:
        ppi_df (pd.DataFrame): DataFrame con interacciones proteicas.
        expression_df (pd.DataFrame): Datos de expresión génica.

    Returns:
        Data: Objeto Data de PyTorch Geometric.
    """
    # Crear lista de nodos y mapping
    nodes = list(expression_df.columns)
    node_map = {gene: i for i, gene in enumerate(nodes)}

    # Crear edge_index
    edges = ppi_df[(ppi_df["protein1"].isin(nodes)) & (ppi_df["protein2"].isin(nodes))]
    edge_index = torch.tensor([
        [node_map[edge["protein1"]], node_map[edge["protein2"]]]
        for _, edge in edges.iterrows()
    ]).t().contiguous()

    # Crear matrix de features
    x = torch.tensor(expression_df.T.values, dtype=torch.float)

    return Data(x=x, edge_index=edge_index)

# -------------------------------
# 5. Pipeline General
# -------------------------------

def main_pipeline():
    """
    Pipeline general para obtener la red PPI, expandirla, extraer datos de expresión y crear el objeto PyG Data.
    """
    # Paso 1: Obtener red PPI
    genes = ["TP53", "PIK3CA", "RB1", "BRCA1", "PTEN", "ATM", "EGFR", "BRAF", "BRCA2", "AKT1",
             "PIK3R1", "KDR", "NF1", "ERBB4", "JAK2", "NOTCH1", "TRRAP", "MET", "ALK", "CDKN2A"]
    ppi_df = get_stringdb_interactions(genes)

    # Paso 2: Expandir red
    expanded_genes = get_stringdb_neighbors(genes)
    expanded_ppi_df = get_stringdb_interactions(expanded_genes)

    # Paso 3: Obtener datos de expresión
    expression_df = get_tcga_expression_data(expanded_genes)

    # Paso 4: Crear objeto PyG Data
    pyg_data = create_pyg_data(expanded_ppi_df, expression_df)
    print("PyTorch Geometric Data:", pyg_data)

if __name__ == "__main__":
    main_pipeline()
